In [130]:
import pandas as pd
import numpy as np
import tensorflow as tf
import nltk


# 0. all words -(nltk)-> all word_tokenize
# 1. all word_tokenize -(nltk)-> remove stop words
# 2. words -(TextVectorization)-> word_index
# 3. words -(glove)-> word_embedding
# 4. word_index : word_embedding 

In [131]:
import contractions

# train_path = 'data/train_HW2dataset.csv'
# val_path = 'data/dev_HW2dataset.csv'
# test_path = 'data/test_HW2dataset.csv'

# df = pd.read_csv(train_path)
# dev_df = pd.read_csv(val_path)
# test_df = pd.read_csv(test_path)

# x_train = df['Utterance']
# print(type(contractions.fix(x_train)))

In [187]:
sample = "Did you really?"
"".join(s for s in sample if s.isalpha() or s in ["!", "?", ".", "'", " "])

'Did you really?'

In [132]:
import nlpaug.augmenter.word as naw
aug = naw.SynonymAug(aug_src='wordnet')
# aug = naw.WordEmbsAug(model_type='glove', model_path='model\_glove.840B.300d\glove.840B.300d.txt',action="substitute")

In [188]:
def expand_contraction(x_train, y_train, duplicate_ratios=None):
    print("in expand")
    if duplicate_ratios is not None:
        for index, sample in enumerate(x_train):
            augmented_sample = aug.augment(sample)
            # print(len(x_train))
            # print(x_train.tail(1))
            tmp_df = pd.DataFrame(augmented_sample)
            x_train = pd.concat([x_train, tmp_df], axis=0)
            y_train.append(y_train[index])

        x_train = x_train.reset_index().drop(["index"], axis=1)[0].values.tolist()
    output = []
    for i, sample in enumerate(x_train):
        sample = contractions.fix(sample).lower().replace("—", " ").replace("-", " ").replace("_", " ")
        output.append("".join(s for s in sample if s.isalpha() or s in ["!", "?", ".", "'", " "]))
        # for j, s in enumerate(sample):
        #     if s.isalpha() or s in ["!", "?", ".", "'", " "]:
        #         continue
        #     else:
        #         print("string", sample, len(sample))
        #         tmp = list(sample)
        #         print("list", tmp, len(tmp))
        #         print("j = ", j)
        #         tmp[j] = " "
        #         sample = "".join(tmp)
    # x_train = x_train.str.lower().str.replace("-", " ").str.replace("_", " ").str.replace("—", " ")
    # x_train = x_train.str.replace(r'[^\w\s]+', '').str.replace('\d+', '')
    # print(x_train)
    print("end expand")
    return output

In [134]:
# train_path = 'data/train_HW2dataset.csv'
# val_path = 'data/dev_HW2dataset.csv'
# test_path = 'data/test_HW2dataset.csv'

# df = pd.read_csv(train_path)

# x_train = df['Utterance']
# y_train = df['Emotion'].values.tolist()



In [189]:
from collections import Counter
def read_data(train_path, val_path, test_path):
    df = pd.read_csv(train_path)
    dev_df = pd.read_csv(val_path)
    test_df = pd.read_csv(test_path)

    y_train = df['Emotion'].values.tolist()
    y_valid = dev_df['Emotion'].values.tolist()

    # compute duplicate ratio
    emotion_label = {"neutral": 0, "anger": 1, "joy": 2, "surprise": 3, "sadness": 4, "disgust": 5, "fear":6}
    duplicate_ratios = {}
    count = Counter()
    for emotion in [y_train, y_valid]:
        count.update(emotion)
    
    for emotion in count:
        max_key = max(count, key=count.get)
        ratio = count[max_key] // count[emotion]
        duplicate_ratios[ emotion_label[emotion] ] = ratio
    print(count)

    # pre-processing & duplicate
    x_train = expand_contraction(df['Utterance'], y_train, duplicate_ratios)

    x_valid = expand_contraction(dev_df['Utterance'].values.tolist(), y_valid, None)
    x_test = expand_contraction(test_df['Utterance'].values.tolist(), None, None)
    print("read data, ok")
    



    return duplicate_ratios, x_train, y_train, x_valid, y_valid, x_test



In [136]:
# train_path = 'data/train_HW2dataset.csv'
# val_path = 'data/dev_HW2dataset.csv'
# test_path = 'data/test_HW2dataset.csv'

# duplicate_ratios, x_train, y_train, x_valid, y_valid, x_test = read_data(train_path, val_path, test_path)
# print(duplicate_ratios)


In [137]:
def emotion_to_label(y):
    labels = np.zeros((len(y), 7))
    emotion_label = {"neutral": 0, "anger": 1, "joy": 2, "surprise": 3, "sadness": 4, "disgust": 5, "fear":6}
    for i in range(len(y)):
        labels[i][ emotion_label[y[i]]] = 1
    return labels


In [138]:
# sample = "hello?! how, are-you54_3(^&$"

# sample = sample.replace("_", " ")
# sample = sample.replace("-", " ")
# sample = sample.replace("—", " ")

# for s in sample:
#     if s.isalnum() or s in ["!", "?", ".", "'", " "]:
#         continue
#     elif s in ["-", "_", "—"]:
#         s = " "
#     else:
#         s = ""
# # sample = ''.join(s for s in sample if  s.isalnum() or (s in ["!", "?", "'", " "]) )

# print(sample)

In [139]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
# stop_words.difference_update(["not", "no", "why", "how"])
print("not" in stop_words)

True


In [140]:
# aug = naw.WordEmbsAug(
#     model_type='glove', model_path='model\_glove.840B.300d\glove.840B.300d.txt',
#     action="substitute")
# augmented_text = aug.augment(text)

In [190]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


def tokenized_remove_stopwords(samples, labels=None, duplicate_ratios=None):
    # neutral -> 0
    if labels is not None:
        labels = emotion_to_label(labels)


    
    # stop_words = set(stopwords.words('english'))
    # stop_words.difference_update({"not", "no", "why", "how"})
    valid_words = set(nltk.corpus.words.words())
    valid_words.update(["!", "?", ".", "'"])
    
    tokenized_samples = []
    duplicate_index = []
    max_cnt = -1
    for i, sample in enumerate(samples):
        cnt = 0
        # for i, s in enumerate(sample):
        #     if s.isalnum() or s in ["!", "?", ".", "'", " "]:
        #         continue
        #     elif s in ["-", "_", "—"]:
        #         sample = " "
        #     else:
        #         s = ""

        word_tokens = word_tokenize(sample)
        
        clean_sample = ""
        for word in word_tokens:
            # if (word.lower() not in stop_words): # (word.lower() in valid_words) : # and 
            clean_sample = clean_sample + " " + word
            cnt += 1
        
        # train, valid mode
        if duplicate_ratios is not None and labels[i][0] != 0:
            duplicate_times = duplicate_ratios[ np.argmax(labels[i]) ]
            for _ in range(duplicate_times):
                duplicate_index.append(i)
        # if i == 4:
        #     print(clean_sample)
        tokenized_samples.append(clean_sample)
        max_cnt = max(max_cnt, cnt)
    print("max cnt = ", max_cnt)
    print("tokenized_remove_stopwords, ok")
    if duplicate_ratios is not None:
        for d_index in duplicate_index:
            augmented_text = aug.augment(tokenized_samples[d_index])
            tokenized_samples.append(augmented_text)
            labels = np.concatenate([labels, np.expand_dims(labels[d_index], axis=0)], axis=0)

    # count = Counter(labels)
    # print(count)
    return tokenized_samples, labels

# train_path = 'data/train_HW2dataset.csv'
# val_path = 'data/dev_HW2dataset.csv'
# test_path = 'data/test_HW2dataset.csv'

# duplicate_ratios, x_train, y_train, x_valid, y_valid, x_test = read_data(train_path, val_path, test_path)
# tokenized_train, label_train = tokenized_remove_stopwords(x_train[1000:], None, duplicate_ratios=None)
# print(tokenized_train[4])

# t_samples = tokenized_remove_stopwords(["I jkdgh not no see happy ?! ."])
# print(t_samples)
# print(tokenized_samples[:2])
# tmp = max([ len(t) for t in tokenized_samples])
# print("max sample word lenth = ", tmp)



In [142]:
from keras.layers import TextVectorization

def word_to_index(tokenized_samples:list, SAMPLE_LENGTH=60):
    vectorizer = TextVectorization(output_sequence_length=SAMPLE_LENGTH)
    text_ds = tf.data.Dataset.from_tensor_slices(tokenized_samples).batch(1024)
    vectorizer.adapt(text_ds)
    print("word_to_index, ok")
    return vectorizer

# vectorizer = word_to_index(tokenized_samples)
# word_index_example = vectorizer(tokenized_samples[1]).numpy()
# print(word_index_example)
# print(vectorizer.get_vocabulary()[166])

In [143]:
def get_word_embedding_dict(glove_size=300):
    word_embedding_dict = {}

    with open(f"model\glove.840B.300d.txt", encoding="utf-8") as f:
        lines = f.readlines()
        for line in lines:
            word, embeddings = line.split(maxsplit=1)
            embeddings = np.fromstring(embeddings, "f", sep=" ")
            
            word_embedding_dict[word] = embeddings

    print("get_word_embedding_dict, ok")
    print("Found %s word vectors." % len(word_embedding_dict))
    return word_embedding_dict

In [144]:
def get_index_embedding_matrix(vectorizer, word_embedding_dict):
    all_sample_words = vectorizer.get_vocabulary()
    word_index_dict = dict(zip(all_sample_words, range(len(all_sample_words))))
    index_embedding_matrix = np.zeros((len(all_sample_words), 300) )
    hit = 0
    miss = 0
    for word, index in word_index_dict.items():
        embedding = word_embedding_dict.get(word)
        if embedding is not None and embedding.shape != (0,):
            # print(f"hit: {word}", embedding.shape)
            
            index_embedding_matrix[index] = embedding
            hit += 1
        else:
            miss += 1
            print(f"miss: {word}")
    print(f"hit = {hit}, miss = {miss}")
    print("get_index_embedding_matrix, ok")
    return all_sample_words, index_embedding_matrix
    
# index_embedding_matrix = get_index_embedding_matrix(all_sample_words, word_index_dict)

In [224]:
from keras.layers import Embedding, Dropout, LSTM, Dense, Bidirectional, Layer, Flatten
from keras import Sequential
from keras.models import Model
from keras_self_attention import SeqSelfAttention
from sklearn.metrics import f1_score, recall_score, precision_score

def train(all_sample_words, embedding_matrix, x_train, y_train, x_valid, y_valid, SAMPLE_LENGTH):
    
    class Metrics(tf.keras.callbacks.Callback):
        def __init__(self, valid_data):
            super(Metrics, self).__init__()
            self.validation_data = valid_data

        def on_epoch_end(self, epoch, logs=None):
            logs = logs or {}
            val_predict = np.argmax(self.model.predict(self.validation_data[0]), -1)
            val_targ = self.validation_data[1]
            if len(val_targ.shape) == 2 and val_targ.shape[1] != 1:
                val_targ = np.argmax(val_targ, -1)

            _val_f1 = f1_score(val_targ, val_predict, average='macro')
            _val_recall = recall_score(val_targ, val_predict, average='macro')
            _val_precision = precision_score(val_targ, val_predict, average='macro')

            logs['val_f1'] = _val_f1
            # logs['val_recall'] = _val_recall
            # logs['val_precision'] = _val_precision
            # print(" — val_f1: %f " % (_val_f1))
            return
    
    
    embedding_layer = Embedding(
        input_dim=len(all_sample_words),
        output_dim=300,
        embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
        input_length=SAMPLE_LENGTH,
        trainable=False,
    )

    
    model = Sequential()
    model.add(embedding_layer)
    model.add(Bidirectional(LSTM(16, dropout=0.2, recurrent_dropout=0.2)))
    # model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
    # model.add(SeqSelfAttention(attention_activation='sigmoid'))
    model.add(Dropout(0.5))
    # model.add(LSTM(16, dropout=0.2, recurrent_dropout=0.2))
    # model.add(SeqSelfAttention(attention_activation='sigmoid'))
    # model.add(Flatten())
    model.add(Dense(7, activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    # model.summary()
    print(np.array(x_train).shape)
    print(np.array(y_train).shape)
    print(np.array(x_valid).shape)
    print(np.array(y_valid).shape)

    ck_callback = tf.keras.callbacks.ModelCheckpoint('./checkpoints/new_balance_{epoch:02d}.pth', verbose=2, save_freq="epoch")
    tb_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs", profile_batch=0)
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    x_valid = np.array(x_valid)
    y_valid = np.array(y_valid)
    model.fit(x_train, y_train, batch_size=128, epochs=200, 
                validation_data=(x_valid, y_valid),
                callbacks=[Metrics(valid_data=(x_valid, y_valid)), ck_callback, tb_callback])
    # model.fit(np.array(x_train), np.array(y_train), batch_size=128, epochs=20, validation_data=(np.array(x_valid), np.array(y_valid)))

    return model


In [146]:
word_embedding_dict = get_word_embedding_dict(glove_size=300)

get_word_embedding_dict, ok
Found 2195885 word vectors.


In [147]:
train_path = 'data/train_HW2dataset.csv'
val_path = 'data/dev_HW2dataset.csv'
test_path = 'data/test_HW2dataset.csv'

duplicate_ratios, x_train, y_train, x_valid, y_valid, x_test = read_data(train_path, val_path, test_path)
tokenized_train, label_train = tokenized_remove_stopwords(x_train, y_train, duplicate_ratios=None)
tokenized_val, label_val = tokenized_remove_stopwords(x_valid, y_valid, None)
tokenized_test, _ = tokenized_remove_stopwords(x_test, None, None)
tokenized_samples = tokenized_train + tokenized_val

SAMPLE_LENGTH = 85
vectorizer = word_to_index(tokenized_samples, SAMPLE_LENGTH=SAMPLE_LENGTH)




Counter({'neutral': 6560, 'joy': 2526, 'anger': 1708, 'surprise': 1700, 'sadness': 1021, 'disgust': 395, 'fear': 392})
in expand
end expand
in expand
end expand
in expand
end expand
read data, ok
max cnt =  79
tokenized_remove_stopwords, ok
max cnt =  42
tokenized_remove_stopwords, ok
max cnt =  50
tokenized_remove_stopwords, ok
word_to_index, ok


In [148]:
all_sample_words, index_embedding_matrix = get_index_embedding_matrix(vectorizer, word_embedding_dict)


miss: 
miss: [UNK]
miss: to
miss: is
miss: in
miss: on
miss: at
miss: or
miss: by
miss: iodin
miss: pheebs
miss: boutros
miss: tayloe
miss: heldi
miss: griscom
miss: sidesplitter
miss: sauti
miss: treeger
miss: thquirt
miss: rosselini
miss: secern
miss: sayornis
miss: rosss
miss: remoray
miss: phoebs
miss: oooooooooooooohhhhhhhhhhh
miss: nngghhh
miss: gerston
miss: mebibyte
miss: zelner
miss: willick
miss: siadic
miss: severalise
miss: scrud
miss: magioni
miss: holierthanthou
miss: goodacre
miss: bapstein
miss: ysee
miss: viosterol
miss: secernate
miss: whoooaa
miss: waxine
miss: velula
miss: testosteroney
miss: tedlock
miss: suity
miss: squatternut
miss: snuffalopagus
miss: schemp
miss: rdtor
miss: ramoray
miss: phobo
miss: phewbedo
miss: phaybobo
miss: nokululu
miss: mwwwooooo
miss: monana
miss: mississ
miss: mashuga
miss: markson
miss: maaaaadd
miss: johnos
miss: janines
miss: hurely
miss: hhiii
miss: gepeto
miss: gellers
miss: dubbies
miss: cupert
miss: cmere
miss: cinderelly
miss:

In [149]:
print(word_embedding_dict["email"])

[]


In [225]:
import warnings 
warnings.filterwarnings("ignore")

# label_train = emotion_to_label(x_train, y_train)
# label_val = emotion_to_label(x_valid, y_valid)
# print(label_val[:2])
vector_train = vectorizer(tokenized_train)
vector_val = vectorizer(tokenized_val)


model = train(all_sample_words, index_embedding_matrix, vector_train, label_train, vector_val, label_val, SAMPLE_LENGTH)
model.save("saved_model.pth")


(25680, 85)
(25680, 7)
(1462, 85)
(1462, 7)
Epoch 1/200
46/46 [==============================] - 1s 21ms/step

Epoch 1: saving model to ./checkpoints\new_balance_01.pth
INFO:tensorflow:Assets written to: ./checkpoints\new_balance_01.pth\assets
201/201 [==============================] - 62s 291ms/step - loss: 1.5984 - accuracy: 0.4511 - val_loss: 1.5925 - val_accuracy: 0.4104 - val_f1: 0.0831
Epoch 2/200
46/46 [==============================] - 1s 21ms/step

Epoch 2: saving model to ./checkpoints\new_balance_02.pth
INFO:tensorflow:Assets written to: ./checkpoints\new_balance_02.pth\assets
201/201 [==============================] - 58s 288ms/step - loss: 1.4815 - accuracy: 0.4742 - val_loss: 1.4991 - val_accuracy: 0.4460 - val_f1: 0.1607
Epoch 3/200
46/46 [==============================] - 1s 21ms/step

Epoch 3: saving model to ./checkpoints\new_balance_03.pth
INFO:tensorflow:Assets written to: ./checkpoints\new_balance_03.pth\assets
201/201 [==============================] - 58s 289ms/s

KeyboardInterrupt: 

In [ ]:
from tensorflow import keras
# Continue training
# 49 -> 63 -> 81
model = keras.models.load_model('checkpoints/not_balance_+28.pth')
model = train(all_sample_words, index_embedding_matrix, vector_train, label_train, vector_val, label_val, SAMPLE_LENGTH)
model.save("saved_model.pth")

In [220]:
# Generate testing results
# 16 63 94
from tensorflow import keras
model = keras.models.load_model('checkpoints/balance_09.pth')
vector_test = vectorizer(tokenized_test)
y_prob = model.predict(np.array(vector_test))
y_pred = np.argmax(y_prob, axis=1)
print(type(y_pred))
print(y_pred.shape)
print(y_pred)
index = range(len(y_pred))
output_data = {"index": index, "emotion": y_pred}
output_df = pd.DataFrame(output_data,)
output_df.to_csv("test_prediction_aug_balance_f31_b128_raw_w_special_e09.csv", index=False)

107/107 [==============================] - 4s 35ms/step
<class 'numpy.ndarray'>
(3400,)
[0 0 0 ... 0 0 0]


In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 91, 300)           1816500   
                                                                 
 bidirectional (Bidirectiona  (None, 256)              439296    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 7)                 1799      
                                                                 
Total params: 2,257,595
Trainable params: 441,095
Non-trainable params: 1,816,500
_________________________________________________________________


In [200]:
model = keras.models.load_model('checkpoints/not_balance_20.pth')


In [207]:
preds = model(vectorizer(np.array([" why are you stupid?"])))
print(preds)

tf.Tensor(
[[0.06853583 0.10446616 0.04617545 0.7259581  0.0259376  0.02282379
  0.00610308]], shape=(1, 7), dtype=float32)


In [213]:
preds = model(vectorizer(np.array(["?"])))
print(preds)

tf.Tensor(
[[0.6206019  0.04904897 0.14757547 0.17219466 0.00637367 0.00204059
  0.00216477]], shape=(1, 7), dtype=float32)
